In [1]:
import ast
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:

# --- 4. SISTEMA DE RECOMENDACIÓN ---


df_peliculas_vectorizadas = pd.read_csv('./archivos/embedding_movies.csv')

# Usamos las variables globales definidas en la parte 1.
# La función ahora solo requiere el user_id.
def parse_embedding_string(embedding_str):
    """Convierte la cadena de texto de embedding a un array de NumPy."""
    try:
        # Usa ast.literal_eval para convertir la cadena de lista/array a una lista de Python
        return np.array(ast.literal_eval(embedding_str), dtype=np.float32)
    except (ValueError, TypeError):
        # Manejo de errores si la cadena está corrupta o vacía
        print(f"Advertencia: No se pudo parsear el embedding. Cadena: {embedding_str[:50]}...")
        # Devuelve un array de ceros del tamaño esperado (1536 para text-embedding-3-small)
        return np.zeros(1536, dtype=np.float32)

df_peliculas_vectorizadas['embedding'] = df_peliculas_vectorizadas['embedding'].apply(parse_embedding_string)

def recomendar_pelicula_a(user_id, top_n=5):

    
    # Referencia a las variables globales (los DataFrames vectorizados)
    # Reemplaza 'df_merged' y 'unique_movies_subset' con los nombres exactos de tus variables globales si son diferentes.
    df_merged=pd.read_csv('./archivos/dataset_merged.csv')
    df_interacciones = df_merged
    #df_peliculas_vectorizadas = pd.read_csv('./archivos/embedding_movies.csv')
    
    # PASO A: Filtrar qué le gusta al usuario (ej: Ratings >= 4)
    user_history = df_interacciones[
        (df_interacciones['userId'] == user_id) & 
        (df_interacciones['rating'] >= 4.0)
    ]
    
    if user_history.empty:
        return f"El usuario {user_id} no tiene suficientes ratings positivos (> 4.0) o no existe en el registro."
    
    liked_movie_ids = user_history['movieId'].values
    
    # PASO B: Recuperar los vectores de esas películas
    liked_vectors_df = df_peliculas_vectorizadas[df_peliculas_vectorizadas['movieId'].isin(liked_movie_ids)]
    
    if liked_vectors_df.empty:
        return f"El usuario {user_id} tiene gustos, pero ninguna de esas películas está en el catálogo vectorizado actual."

    # PASO C: Crear Vector de Usuario (Promedio)
    matrix_liked = np.stack(liked_vectors_df['embedding'].values)
    user_profile_vector = np.mean(matrix_liked, axis=0).reshape(1, -1)
    
    # PASO D: Calcular Similitud con TODO el catálogo disponible
    catalog_matrix = np.stack(df_peliculas_vectorizadas['embedding'].values)
    
    # Coseno entre [Perfil Usuario] vs [Catálogo]
    similarities = cosine_similarity(user_profile_vector, catalog_matrix)
    
    recommendations = df_peliculas_vectorizadas.copy()
    recommendations['similarity_score'] = similarities[0]
    
    # PASO E: Filtrar (Quitar las que ya vio) y Ordenar
    ids_vistos = df_interacciones[df_interacciones['userId'] == user_id]['movieId'].values
    
    recs_finales = recommendations[~recommendations['movieId'].isin(ids_vistos)]
    
    # Ordenar por similitud descendente y tomar Top N
    top_recs = recs_finales.sort_values(by='similarity_score', ascending=False).head(top_n)
    
    return top_recs[['title', 'vote_average', 'similarity_score','genres','overview']],user_history



In [9]:
df_merged=pd.read_csv('./archivos/dataset_merged.csv')
mi_usuario_a_probar = df_merged['userId'].iloc[1900] # Tomando el primer usuario del DF

resultados,user_history = recomendar_pelicula_a(mi_usuario_a_probar)

print(f"\n🎬 Recomendaciones para Usuario {mi_usuario_a_probar}")
if isinstance(resultados, str):
    print(resultados)
else:
    print(resultados.to_markdown(index=False))


🎬 Recomendaciones para Usuario 641
| title                |   vote_average |   similarity_score | genres                                 | overview                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|:---------------------|---------------:|-------------------:|:---------------------------------------|:-------------------------------------------------------

In [4]:
user_history

,userId,movieId,rating,title,overview,genres,vote_average
15,281,1371,4.0,Rocky III,"Now the world champion, Rocky Balboa is living...",['Drama'],6.6
2354,281,296,4.5,Terminator 3: Rise of the Machines,It's been 10 years since John Connor saved Ear...,"['Action', 'Thriller', 'Science Fiction']",5.9
3985,281,480,4.0,Monsoon Wedding,From an exciting Indian wedding comes a relati...,"['Comedy', 'Drama', 'Romance']",6.8
4468,281,509,4.5,Notting Hill,The British comedy from director Roger Michell...,"['Romance', 'Comedy', 'Drama']",7.0
4618,281,527,5.0,Once Were Warriors,A drama about a Maori family lving in Auckland...,['Drama'],7.6
5635,281,590,4.0,The Hours,"""The Hours"" is the story of three women search...",['Drama'],7.0
6077,281,593,4.0,Solaris,Ground control has been receiving strange tran...,"['Drama', 'Science Fiction', 'Adventure', 'Mys...",7.7
7590,281,2028,4.0,Say Anything...,A budding romance between noble underachiever ...,"['Comedy', 'Drama', 'Romance']",7.2
8678,281,260,4.0,The 39 Steps,"While on vacation in London, Canadian Richard ...","['Action', 'Thriller', 'Mystery']",7.4
9905,281,858,4.0,Sleepless in Seattle,A young boy who tries to set his dad up on a d...,"['Comedy', 'Drama', 'Romance']",6.5
